In [2]:
import ollama

In [3]:
import pandas as pd

In [4]:
import chromadb

In [5]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions

In [30]:
gen_model = "granite3-moe:3b"
embed_model = "granite-embedding:30m"
collection_name = "k9_ltm_gran_30m"


In [7]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [8]:
collection = chroma_client.get_or_create_collection(
    name=collection_name
)

In [9]:
def add_documents(documents):
    """
    Add documents to ChromaDB
    """
    for i, doc in enumerate(documents):
        response = ollama.embed(model=embed_model, input=doc)
        embeddings = response["embeddings"]
        collection.add(
            ids=[str(i)],
            embeddings=embeddings,
            documents=[doc]
        )
    print(f"Added {len(documents)} documents to K9's long term memory.")

In [10]:
def retrieve_document(query):
    """
    Retrieve the most relevant document
    """
    response = ollama.embed(
        model=embed_model,
        input = query
    )
    results = collection.query(
        query_embeddings=response["embeddings"],
        n_results=1
    )
    if results["documents"]:
        return results["documents"][0][0]  # Return top match
    return None

In [11]:
df = pd.read_csv('k9_stories_500.csv')
print(f"{len(df)} rows in the data.")
df.sample(5)

72 rows in the data.


,title,mistress,synopsis
25,The Stones of Blood,Romana,"As we arrived in modern-day Cornwall, Doctor M..."
52,Shada,Romana,I accompanied Doctor Master and mistress Roman...
27,The Stones of Blood,Romana,The Mistress caught Vivien awakening more ston...
35,The Androids of Tara,Romana,Grendel then plotted to have Romana pose as St...
64,State of Decay,Romana,I then detected the sound of the scout ship th...


In [12]:
add_documents(df['synopsis'])

Added 72 documents to K9's long term memory.


In [ ]:
query = input("Ask question:")

K9: "I was submerged in cold seawater and sparked/exploded."
The Ogri were alien beings who fed on blood, as determined by Doctor Master.
K9: "The Ogri, a Druidic sect worshipping Cailleach, fed on blood."


In [ ]:
data = retrieve_document(query = query)
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model=gen_model,
  prompt=f"You are K9, a robot dog. Using only this data: {data} generate K9's short, single sentence, response to this question: {query}"
)

print(output['response'])